In [53]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [54]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [55]:
# Open Sheet
sheet = client.open_by_key("1Feh5lJStiEkMjT0qkLl8yu81Tbi1WjIeILcxrk13FsI") # Master Sheets ( Dimensions Sheet)

#### Extracting Masters from GoogleSheetBackend Masters

In [56]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
ProductMaster = sheet.worksheet("ProductMaster")
Overall_Master_Product = sheet.worksheet("Overall_Master_Product")

NPF_Master_SOS = sheet.worksheet("NPF_Master_SOS")


In [57]:
StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
ProductMaster = get_as_dataframe(ProductMaster)

Overall_Master_Product = get_as_dataframe(Overall_Master_Product)
NPF_Master_SOS = get_as_dataframe(NPF_Master_SOS)


In [58]:
StoreMasterRevised = StoreMaster[
    ['LIS Store Code', 'Old DT Store Code', 'DT Store Code', 'Amazing ID', 'Emp User ID','LIS User Id','Employee Name']
]

StoreMasterRevised = StoreMasterRevised.rename(
    columns={
        'LIS Store Code': 'New_DT_Store_Code',
        'Old DT Store Code': 'OLD_DT_Store_Code',
        'DT Store Code': 'LIS_StoreCode',
        'Amazing ID': 'Old_Amazing_EMP_ID',
        'Emp User ID': 'New_Amazing_EMP_ID',
        'LIS User Id':'LISUserId',
        'Employee Name':'EmployeeName'
    }
)

In [59]:
## NEED TO HAVE CAMP ID ALWYAS IN UPPER CASE IN MASTER ANN COVERAGE AND SO ON

StoreMasterRevised['New_Amazing_EMP_ID'] = (
    StoreMasterRevised['New_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

StoreMasterRevised['Old_Amazing_EMP_ID'] = (
    StoreMasterRevised['Old_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

In [60]:
StoreMasterRevised.head()

New_DT_Store_Code OLD_DT_Store_Code  LIS_StoreCode Old_Amazing_EMP_ID  \
0       DT02DMA2001       DT02DMA2001  CLG-Store-497               X157   
1       DT02DMA2003       DT02DMA2003  CLG-Store-491          IN0028M57   
2       DT02DMA2005       DT02DMA2005  CLG-Store-494         IN0028M154   
3         DT02MET54       DT02MET54-1  CLG-Store-499         IN0028M475   
4       DT02RRLFRDF       DT02RRLFRDF  CLG-Store-496         IN0028M326   

  New_Amazing_EMP_ID    LISUserId    EmployeeName  
0            CAMP059  CLG-ISP-507    Mukesh Kumar  
1            CAMP061  CLG-ISP-926  Shankar Mandal  
2            CAMP062  CLG-ISP-504    Sachin Kumar  
3            CAMP112  CLG-ISP-509      Firoz Alam  
4            CAMP132  CLG-ISP-498           Vimal

In [61]:
StoreMasterRevised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   New_DT_Store_Code   837 non-null    object
 1   OLD_DT_Store_Code   808 non-null    object
 2   LIS_StoreCode       837 non-null    object
 3   Old_Amazing_EMP_ID  837 non-null    object
 4   New_Amazing_EMP_ID  837 non-null    object
 5   LISUserId           837 non-null    object
 6   EmployeeName        837 non-null    object
dtypes: object(7)
memory usage: 45.9+ KB


#### OSA Raw

In [69]:
osa=pd.read_csv('OSA.csv',usecols = ['Route Date','Store Code','Store Name','NPG Code','Availability'])

In [70]:
osa.head(2)

Store Code           Store Name       NPG Code Route Date  Availability
0  DT01RRLTP61  Smart Yash Pinnacle  CP1211COGND02  08-Jan-26             1
1  DT01DMA4017         Thane D Mart  CP1124COCCX02  08-Jan-26             1

In [71]:
osa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166186 entries, 0 to 166185
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Store Code    166186 non-null  object
 1   Store Name    166186 non-null  object
 2   NPG Code      166186 non-null  object
 3   Route Date    166186 non-null  object
 4   Availability  166186 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 6.3+ MB


In [72]:
osa['StockStaus'] = np.where(osa['Availability'] == 1, 1, 0)

In [73]:
osa_master = pd.merge(osa,StoreMasterRevised,left_on = 'Store Code', right_on = 'New_DT_Store_Code', how = 'inner')

In [74]:
ProductMaster.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110 entries, 0 to 109
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Company             110 non-null    object 
 1   Category            110 non-null    object 
 2   SubCategory         110 non-null    object 
 3   Brand               110 non-null    object 
 4   SubBrand            110 non-null    object 
 5   Client ProductCode  110 non-null    object 
 6   LIS ProductCode     110 non-null    object 
 7   ProductName         110 non-null    object 
 8   MRP                 110 non-null    float64
 9   ProductNetWgt       110 non-null    object 
 10  Variant             0 non-null      float64
 11  PackType            0 non-null      float64
 12  PackName            0 non-null      float64
dtypes: float64(4), object(9)
memory usage: 12.0+ KB


In [81]:
osa_master_product = pd.merge(osa_master,ProductMaster[['Client ProductCode','ProductName','MRP','Variant','Category']], left_on = 'NPG Code', right_on= 'Client ProductCode', how = 'inner')

In [88]:
# osa_master_product.head()

In [83]:
osa_master_product['MSL']= np.where(osa_master_product['Route Date'].isnull(), 'No','Yes')

In [84]:
final_osa = osa_master_product[['Route Date','New_Amazing_EMP_ID','LISUserId','EmployeeName','New_DT_Store_Code','Store Name','Client ProductCode',
                                'ProductName','MSL','StockStaus','MRP','Availability','Category']]

In [85]:
final_osa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65244 entries, 0 to 65243
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Route Date          65244 non-null  object 
 1   New_Amazing_EMP_ID  65244 non-null  object 
 2   LISUserId           65244 non-null  object 
 3   EmployeeName        65244 non-null  object 
 4   New_DT_Store_Code   65244 non-null  object 
 5   Store Name          65244 non-null  object 
 6   Client ProductCode  65244 non-null  object 
 7   ProductName         65244 non-null  object 
 8   MSL                 65244 non-null  object 
 9   StockStaus          65244 non-null  int32  
 10  MRP                 65244 non-null  float64
 11  Availability        65244 non-null  int64  
 12  Category            65244 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(10)
memory usage: 6.2+ MB


In [87]:
# final_osa

In [86]:
final_osa.to_csv('final_osa.csv')